In [99]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup

In [94]:
def details_scrape(url):
    """
    Uses selenium to scrape additional information contained in a sub-menu
    
    Clicks the first box, guru throws up a log-in, goes back, opens all the boxes
    and then finishes by extracting all the html from each box.
    
    This takes in a single URL (string) and extracts from that. One problem that may arise
    is if it doesn't throw the log-in box and the driver tries to go back a page.
    However, I think we can get around this by just loading each URL new. 
    
    May be more efficient to page through? That's for another time.
    
    RETURNS: list of names and list of HTML for each details box.
    
    Test URL:
    "https://www.guru.com/d/freelancers/lc/united-states/california/los-angeles/pg/1/"
    """
    
    driver = webdriver.Firefox()
    driver.get(url)
    login_form = driver.find_elements_by_xpath('//button[@class="tabControls__button"]')

    time.sleep(2) # Wait to click
    login_form[2].click()

    time.sleep(2) # Wait for log-in prompt
    driver.back() # Go back to main page
    time.sleep(2) # Wait to scrape

    # Pulling the paths again (they change for some reason)
    login_form = driver.find_elements_by_xpath('//button[@class="tabControls__button"]') 
    counter = 2
    
    # There are multiple buttons so here I am just clicking the ones I want (2,6,10,...)
    for i, val in enumerate(login_form):    
        if i == counter:
            login_form[i].click()
            counter += 4
    
    print("Completed opening all the tabs")

    # Pull elements in the now opened detail boxes
    user_name = driver.find_elements_by_xpath('//h3[@class="freelancerAvatar__screenName"]')
    user_detail = driver.find_elements_by_xpath('//ul[@class="feedback__stats clearfix"]')

    # Extract text from the names and HTML from the details
    # Will parse the detail_html further with beautiful soup
    names = []
    for i, val in enumerate(user_name):
        names.append(val.text)

    detail_html = []
    for i, val in enumerate(user_detail):
        detail_html.append(val.get_attribute('innerHTML'))

    driver.close()
    print("Finished")
    
    return names, detail_html

In [ ]:
def details_to_soup(x):
    
    soups = []
    for i, val in enumerate(x):
        soup.append(BeautifulSoup(x,'html.parser'))

In [ ]:
def 

In [105]:
# Building beautiful soup to process detail
detail = BeautifulSoup(test[1][1], 'html.parser')
print(detail.prettify())

<li class="feedback__stats__stat">
 Member since:
 <em>
  Apr, 2019
 </em>
</li>
<li class="feedback__stats__stat">
 Earnings in Past 12 Months:
 <em>
  $23,618
 </em>
</li>
<li class="feedback__stats__stat">
 All Time Earnings:
 <em>
  $23,618
 </em>
</li>
<li class="feedback__stats__stat">
 Employers:
 <em>
  1
 </em>
</li>
<li class="feedback__stats__stat">
 Invoices Paid:
 <em>
  24
 </em>
</li>
<li class="feedback__stats__stat">
 Largest Employer:
 <em>
  $23,618
 </em>
</li>


In [115]:
print(detail.find_all('em'))

[<em> Apr, 2019</em>, <em> $23,618 </em>, <em>$23,618 </em>, <em>1 </em>, <em>24 </em>, <em> $23,618 </em>]
